In [12]:
import numpy as np
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.layers import concatenate
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from networks_lib import *
import h5py

import sys
sys.path.insert(0, '~/DATA1/Big_Sample/NoteBooks/')

#import networks_lib
#from keras.layers import Input,Dense, BatchNormalization, Dropout, Conv2D, MaxPooling2D, Activation, Flatten, Concatenate
#from keras import layers

import tensorflow as tf

import keras.backend as K


config = tf.ConfigProto()

#Prende al massimo la memoria necessaria
config.gpu_options.allow_growth = True

#Limita alla frazione di memoria totale
config.gpu_options.per_process_gpu_memory_fraction = 0.3

#Dice di utilizzare la configurazione che abbiamo appena settato
K.tensorflow_backend.set_session(tf.Session(config=config))

import ROOT as r

r.enableJSVis()

2019-06-07 09:31:09.399775: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1471] Adding visible gpu devices: 0, 1
2019-06-07 09:31:09.399857: I tensorflow/core/common_runtime/gpu/gpu_device.cc:952] Device interconnect StreamExecutor with strength 1 edge matrix:
2019-06-07 09:31:09.399874: I tensorflow/core/common_runtime/gpu/gpu_device.cc:958]      0 1 
2019-06-07 09:31:09.399888: I tensorflow/core/common_runtime/gpu/gpu_device.cc:971] 0:   N N 
2019-06-07 09:31:09.399900: I tensorflow/core/common_runtime/gpu/gpu_device.cc:971] 1:   N N 
2019-06-07 09:31:09.400556: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1084] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 3656 MB memory) -> physical GPU (device: 0, name: TITAN Xp, pci bus id: 0000:21:00.0, compute capability: 6.1)
2019-06-07 09:31:09.410168: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1084] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:1 with 1517 MB memory) -

In [2]:
img_rows, img_cols, nb_channels = 32, 32, 2        
input_dir = '.'
decays = ['SingleElectronPt50_IMGCROPS_n249k_RHv1_copy', 'SinglePhotonPt50_IMGCROPS_n249k_RHv1_copy']

def load_data(decays, start, stop):
    global input_dir
    dsets = [h5py.File('%s/%s.hdf5'%(input_dir,decay)) for decay in decays]
    X = np.concatenate([dset['/X'][start:stop] for dset in dsets])
    y = np.concatenate([dset['/y'][start:stop] for dset in dsets])
    assert len(X) == len(y)
    return X, y

In [3]:
images, targets= load_data(decays, 0, -1)


In [4]:
order= np.asarray([i for i in range(len(targets))])
np.random.shuffle(order)
targets_temp=[]
images_temp=[]
for i in range(len(targets)):
    targets_temp.append(targets[order[i]])
    images_temp.append(images[order[i]])
images=np.asarray(images_temp)
targets=np.asarray(targets_temp)
    
    
    

In [5]:
Ntrain=int(0.70*len(images))
Nval=int(0.13*len(images))
images_train=images[:Ntrain]
target_train=targets[:Ntrain]
images_val=images[Ntrain:Ntrain+Nval]
target_val=targets[Ntrain:Ntrain+Nval]
images_test=images[Ntrain+Nval:]
target_test=targets[Ntrain+Nval:]


# Actual net


In [79]:
d=0.2
dn=1024
input_images=Input(shape=images_train[0].shape)
conv=BatchNormalization(name="conv_BN_input")(input_images)
conv=Conv2D(16, kernel_size=(2,2), padding="same", name="conv_Conv_1")(conv)
#conv=Conv2D(32, kernel_size=(4,2), padding="same", name="conv_Conv_1_2")(conv)
conv=BatchNormalization(name="conv_BN_1")(conv)
conv=Dropout(d, name="conv_Do_1")(conv)
conv=Activation("relu", name="conv_Act_1")(conv)
conv=MaxPooling2D((2,2), name="conv_MP_1")(conv)

conv=Conv2D(32, kernel_size=(2,2), padding="same", name="conv_Conv_2")(conv)
#conv=Conv2D(64, kernel_size=(2,2), padding="same", name="conv_Conv_2_2")(conv)
conv=BatchNormalization(name="conv_BN_2")(conv)
conv=Dropout(d, name="conv_Do_2")(conv)
conv=Activation("relu", name="conv_Act_2")(conv)

conv=Conv2D(64, kernel_size=(2,2), padding="same", name="conv_Conv_3")(conv)
conv=BatchNormalization(name="conv_BN_3")(conv)
conv=Dropout(d, name="conv_Do_3")(conv)
conv=Activation("relu", name="conv_Act_3")(conv)

conv=Flatten()(conv)
conv=Dense(dn, name="conv_Dense_1")(conv)
conv=BatchNormalization(name="conv_BN_4")(conv)
conv=Dropout(d, name="conv_Do_4")(conv)
conv=Activation("relu", name="conv_Act_4")(conv)

conv=Dense(dn, name="conv_Dense_2")(conv)
conv=BatchNormalization(name="conv_BN_5")(conv)
conv=Dropout(d, name="conv_Do_5")(conv)
conv=Activation("relu", name="conv_Act_5")(conv)
conv=Dense(1, activation="sigmoid", name="conv_Dense_3")(conv)

model=Model(inputs=[input_images], outputs=conv)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        (None, 32, 32, 2)         0         
_________________________________________________________________
conv_BN_input (BatchNormaliz (None, 32, 32, 2)         8         
_________________________________________________________________
conv_Conv_1 (Conv2D)         (None, 32, 32, 16)        144       
_________________________________________________________________
conv_BN_1 (BatchNormalizatio (None, 32, 32, 16)        64        
_________________________________________________________________
conv_Do_1 (Dropout)          (None, 32, 32, 16)        0         
_________________________________________________________________
conv_Act_1 (Activation)      (None, 32, 32, 16)        0         
_________________________________________________________________
conv_MP_1 (MaxPooling2D)     (None, 16, 16, 16)        0         
__________

In [80]:
LR_ST=1e-2
LR_END=1e-4
EPOCHS=240
LR_SCHEDULER = LearningRateScheduler(lambda epoch: LR_ST * (LR_END/LR_ST)** (epoch/EPOCHS))
filepath = 'hackaton_5.h5'
SAVE_W = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
model.compile(loss='binary_crossentropy',
                optimizer=Adam(lr=LR_ST),
                metrics=['acc'])



In [ ]:
history=model.fit([images_train], target_train ,
                    epochs=EPOCHS, batch_size=128, 
                    validation_data=([images_val], target_val),
                   callbacks=[LR_SCHEDULER, SAVE_W])

Train on 348598 samples, validate on 64739 samples
Epoch 1/240
221440/348598 [==================>...........] - ETA: 28s - loss: 0.6728 - acc: 0.6173

In [14]:
hist_n=0
from array import array
val_loss=array('d', history.history['val_loss'])
train_loss=array('d', history.history['loss'])

epochs=array('d', range(len(val_loss)))
loss_graph=r.TMultiGraph("loss_graph", "loss_graph")
val_loss_graph=r.TGraph(len(epochs), epochs, val_loss)
train_loss_graph=r.TGraph(len(epochs), epochs, train_loss)

val_loss_graph.SetMarkerColor(r.kRed)
val_loss_graph.SetMarkerStyle(20)
val_loss_graph.SetMarkerSize(0.4)

train_loss_graph.SetMarkerColor(r.kBlue)
train_loss_graph.SetMarkerStyle(20)
train_loss_graph.SetMarkerSize(0.4)

loss_graph.Add(train_loss_graph)
loss_graph.Add(val_loss_graph)

c=r.TCanvas()
legend=r.TLegend(0.7, 0.7, 0.85, 0.85)
legend.AddEntry(train_loss_graph, "train loss")
legend.AddEntry(val_loss_graph, "val loss")
#legend.SetTextFont(20)
#legend.SetTextSize(1)
#r.gPad.Update()
c.Draw()
loss_graph.Draw("apl")
legend.Draw()

In [81]:
model.load_weights('hackaton_5.h5')
evaluations=model.evaluate(images_test, target_test)

84661/84661 [==============================] - 17s 205us/step


In [82]:
evaluations

[0.59498543642429, 0.7134808235253167]

In [19]:
predictions=model.predict([images_test])

# Plots

In [21]:
ph=r.TH1F("ph","ph", 100, 0, 1)
el=r.TH1F("el","el", 100, 0, 1)
for i in range(len(predictions)):
    if target_test[i]==0:
        ph.Fill(predictions[i])
    else:
        el.Fill(predictions[i])
c=r.TCanvas()
el.SetLineColor(r.kRed)
ph.Draw()
el.Draw("same")
#c.GetYaxis().SetRangeUser()
c.Draw()


Warning in <TROOT::Append>: Replacing existing TH1: ph (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: el (Potential memory leak).


In [41]:
x=[]
y=[]
cut_step=0.005
ROC=r.TMultiGraph("ROC", "ROC")
myROC=r.TGraphAsymmErrors(int(1/cut_step))
Npoint=int(1/cut_step)
el_passed=r.TH1I("el_passed","",Npoint, 0, Npoint)
el_total=r.TH1I("el_total","",Npoint, 0, Npoint)
ph_passed=r.TH1I("ph_passed","",Npoint, 0, Npoint)
ph_total=r.TH1I("ph_total","",Npoint, 0, Npoint)
for t in range(Npoint):
    el_eff=0
    ph_eff=0
    el_tot=0
    ph_tot=0
    cut=cut_step*t
    
    for i, pred in enumerate(predictions):
        if target_test[i]==1:
            el_tot+=1
            if pred>=cut:
                el_eff+=1
        elif target_test[i]==0:
            ph_tot+=1
            if pred>=cut:
                ph_eff+=1
    el_passed.SetBinContent(t+1, el_eff)
    ph_passed.SetBinContent(t+1, ph_eff)
    el_total.SetBinContent(t+1, el_tot)
    ph_total.SetBinContent(t+1, ph_tot)
    
    el_eff/=el_tot
    ph_eff/=ph_tot
    myROC.SetPoint(t, el_eff, (1-ph_eff))
    x.append(el_eff)
    y.append(1-ph_eff)
            
el_TEfficiency=r.TEfficiency(el_passed, el_total)
ph_TEfficiency=r.TEfficiency(ph_passed, ph_total)
for i in range(Npoint):
    el_error_low=el_TEfficiency.GetEfficiencyErrorLow(i+1)
    el_error_high=el_TEfficiency.GetEfficiencyErrorUp(i+1)
    ph_error_low=ph_TEfficiency.GetEfficiencyErrorLow(i+1)
    ph_error_high=ph_TEfficiency.GetEfficiencyErrorUp(i+1)
    myROC.SetPointError(i+1, el_error_low, el_error_high, ph_error_low, ph_error_high)
    



Warning in <TROOT::Append>: Replacing existing TH1: hs_passed (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hs_total (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: pu_passed (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: pu_total (Potential memory leak).


In [48]:
ROC.SetTitle("; el Efficiency; ph Rejection")
myROC.SetMarkerStyle(11)
myROC.SetMarkerSize(1)
myROC.SetMarkerColor(3)

ROC.Add(myROC)
ROC.GetXaxis().SetRangeUser(0,1)
ROC.GetYaxis().SetRangeUser(0,1)

c=r.TCanvas("c", "c", 800, 800)
ROC_legend=r.TLegend(0.59, 0.75, 0.74, 0.85)
ROC_legend.SetLineColor(0)
ROC_legend.SetFillColor(r.kNone)
ROC_legend.AddEntry(myROC, "ROC AUC=0.7644")

ROC.GetXaxis().SetNdivisions(1020)
ROC.GetYaxis().SetNdivisions(1010)
c.SetGrid()
c.SetTicks(1,1)
c.Update()
c.Draw()
ROC.Draw("ape")
ROC_legend.Draw()


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c


In [46]:
myROC.SetPoint(len(x), 0, 0)

In [49]:
roc_auc=myROC.Integral()
print("ROC area under curve: "+str(roc_auc))

ROC area under curve: 0.764448806900384
